In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
pat_df=pd.read_csv('../input/PATIENTS.csv',low_memory=False)
pat_df.head()

In [ ]:
pres_df=pd.read_csv('../input/PRESCRIPTIONS.csv',low_memory=False)
pres_df.head()

In [ ]:
note_df=pd.read_csv('../input/NOTEEVENTS.csv',low_memory=False)
note_df.head()

In [ ]:
# print("Total no. of rows in this csv:{}".format(note_df.shape[0]))
# print("Different types of category in this csv are:\n{}".format(note_df.CATEGORY.value_counts()))
# print("Different types of description in this csv are:\n{}".format(note_df.DESCRIPTION.value_counts()))
# print("Unique patient ids and their number of arrivals to the hospital are:\n{}".format(note_df.SUBJECT_ID.value_counts()))

In [ ]:
print("Number of unique patients:{}".format(len(note_df.SUBJECT_ID.unique())))
print("Number of patients who have visited the hospital only once:{}".format(note_df.shape[0]-len(note_df[note_df.duplicated(['SUBJECT_ID'],keep=False)])))

In [ ]:
#word counts
note_df['word_count'] = note_df['TEXT'].apply(lambda x: len(str(x).split(" ")))
note_df['word_count'].describe()

In [ ]:
train_des = note_df['TEXT'].fillna("not available").values
#train_subid=req_df['SUBJECT_ID'].values


In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
#from nltk.stem import WordNetLemmatizer 
  
#lemmatizer = WordNetLemmatizer() 
stop_words = set(stopwords.words('english')) 

def data_clean(big_list):
    for i in range(len(big_list)):
        curr_string=big_list[i]
        curr_string=curr_string.lower()
        word_tokens = word_tokenize(curr_string)
        filtered_sentence = [] 
        for w in word_tokens: 
            if (w not in stop_words) and (w.isalpha()):
                #word=lemmatizer.lemmatize(w)
                filtered_sentence.append(w)
        big_list[i]=' '.join(filtered_sentence)
    return big_list

In [ ]:
train_des=data_clean(train_des)


In [ ]:
df=pd.DataFrame({'TEXT':train_des})
df['word_count'] = df['TEXT'].apply(lambda x: len(str(x).split(" ")))
df['word_count'].describe()

In [ ]:
df.to_csv('text_cleaned.csv',index=False)

In [ ]:
from keras.layers import Dense, Input, Dropout, Activation
from keras.layers import Conv2D,GlobalMaxPooling2D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Dense,merge,Lambda,Flatten
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
import keras.backend as K

dim=300
filter_size=32
ksize=5

#Model1
inp1 = Input(shape=(None,dim,1))
x1 = Conv2D(filter_size, kernel_size=(ksize,dim) ,padding='valid', kernel_initializer='normal')(inp1)
x2 = GlobalMaxPooling2D()(x1)

#Model2
inp2 = Input(shape=(None,dim,1))
y1 = Conv2D(filter_size, kernel_size=(ksize,dim) ,padding='valid', kernel_initializer='normal')(inp2)
y2 = GlobalMaxPooling2D()(y1)

#Model3
merged = concatenate([x2,y2])
out = Dense(10, activation="softmax")(merged)
model = Model(inputs=[inp1,inp2], outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#Checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [ ]:
from nltk.tokenize import word_tokenize 

def data_list(big_list):
    data_list=[]
    for i in range(len(big_list)):
        curr_string=big_list[i]
        curr_string=curr_string.lower()
        word_tokens = word_tokenize(curr_string)
        data_list.append(word_tokens) 
    return data_list


from gensim.models import Word2Vec

# train model
new_list=data_list(train_des)
model = Word2Vec(new_list,size=50, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(len(words))
# access vector for one word
print(model['admission'])
# save model
model.save('model-mimic.bin')
# load model
new_model = Word2Vec.load('model-mimic.bin')
#print(new_model)